# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import tensorflow as tf
# from tensorflow.keras import layers
import gensim
import sklearn
from sklearn.model_selection import train_test_split


2022-11-09 01:21:35.776699: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loading the dataset

In [2]:
import pickle 

with open('news_final_dataset.pickle', 'rb') as file:
    news = pickle.load(file)

In [3]:
X_train, X_test, y_train, y_test = news['X_train'], news['X_test'], news['y_train'], news['y_test']

In [4]:
X_train.columns

Index(['title', 'text', 'subject', 'date', 'all_text', 'char_count',
       'word_count', 'sent_count', 'capital_word_count', 'quoted_word_count',
       'stopword_count', 'unique_word_count', 'punct_count', 'avg_wordlength',
       'avg_sentlength', 'unique_vs_words', 'stopwords_vs_words', 'noun_count',
       'adverb_count', 'sentiment_score', 'title_vect', 'text_vect',
       'all_text_vect', 'char_count', 'word_count', 'sent_count',
       'capital_word_count', 'quoted_word_count', 'stopword_count',
       'unique_word_count', 'punct_count', 'avg_wordlength', 'avg_sentlength',
       'unique_vs_words', 'stopwords_vs_words', 'noun_count', 'adverb_count',
       'sentiment_score'],
      dtype='object')

# 1. Metadata and Sentiments - title only

In [5]:
X_train_title = X_train.iloc[:,23:]
X_test_title = X_test.iloc[:,23:]
y_train_title = y_train
y_test_title = y_test

In [6]:
X_train_title.head()

,char_count,word_count,sent_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words,noun_count,adverb_count,sentiment_score
30958,71,12,1,0,0,3,12,0,5.916667,12.0,1.0,0.250000,4,0,0.6712
3203,72,10,1,2,0,0,10,1,7.200000,10.0,1.0,0.000000,3,0,0.0000
11208,127,21,2,3,0,3,21,4,6.047619,10.5,1.0,0.142857,11,1,0.5613
4923,81,13,1,1,0,1,13,2,6.230769,13.0,1.0,0.076923,4,1,-0.2839
14120,103,17,1,2,0,0,17,2,6.058824,17.0,1.0,0.000000,7,0,-0.8056


In [7]:
y_train_title.head()

30958    0
3203     1
11208    1
4923     1
14120    1
Name: fake, dtype: int64

### 1.1 Logistic Regression

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit


lr_clf_title = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(random_state=1))
])

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=1)
lr_cv_results_title = cross_validate(lr_clf_title, X_train_title, y_train_title, cv=cv, return_train_score=True)

In [9]:
lr_cv_results_title

{'fit_time': array([0.22749281, 0.22252488, 0.22786999, 0.20395589, 0.19286704,
        0.16687298, 0.21369004, 0.17273688, 0.22287321, 0.16888499]),
 'score_time': array([0.0049932 , 0.00378609, 0.00356507, 0.00461602, 0.00384402,
        0.0034833 , 0.00571203, 0.00344992, 0.00422072, 0.00363302]),
 'test_score': array([0.94002586, 0.94794698, 0.94406725, 0.94148076, 0.94180407,
        0.94358228, 0.94115745, 0.94228904, 0.94083414, 0.9442289 ]),
 'train_score': array([0.9434519 , 0.94135004, 0.94232013, 0.94320938, 0.94288601,
        0.94211803, 0.94316896, 0.94292643, 0.94292643, 0.94236055])}

In [10]:
print(f'Time for fitting classifier on the train set: {lr_cv_results_title["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {lr_cv_results_title["score_time"].mean()}')
print(f'Accuracy of Train: {lr_cv_results_title["train_score"].mean()}')
print(f'Accuracy of Validation: {lr_cv_results_title["test_score"].mean()}')

Time for fitting classifier on the train set: 0.20197687149047852
Time for scoring classifier on the validation set: 0.004130339622497559
Accuracy of Train: 0.9426717865804365
Accuracy of Validation: 0.9427416747494343


### 1.2 Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import Normalizer

gnb_clf_title = Pipeline([
    ('scale', Normalizer()),
    ('clf', GaussianNB())
])

gnb_cv_results_title = cross_validate(gnb_clf_title, X_train_title, y_train_title, cv=cv, return_train_score=True)

In [ ]:
gnb_cv_results_title

In [ ]:
print(f'Time for fitting classifier on the train set: {gnb_cv_results_title["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {gnb_cv_results_title["score_time"].mean()}')
print(f'Accuracy of Train: {gnb_cv_results_title["train_score"].mean()}')
print(f'Accuracy of Validation: {gnb_cv_results_title["test_score"].mean()}')

### 1.3 SVM

In [ ]:
from sklearn.svm import SVC

svm_clf_title = Pipeline([
    ('scale', StandardScaler()),
    ('clf', SVC(random_state=1))
])

svm_cv_results_title = cross_validate(svm_clf_title, X_train_title, y_train_title, cv=cv, return_train_score=True)

In [ ]:
svm_cv_results_title

In [ ]:
print(f'Time for fitting classifier on the train set: {svm_cv_results_title["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {svm_cv_results_title["score_time"].mean()}')
print(f'Accuracy of Train: {svm_cv_results_title["train_score"].mean()}')
print(f'Accuracy of Validation: {svm_cv_results_title["test_score"].mean()}')

### 1.4 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf_title = Pipeline([
    ('scale', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=1))
])

rf_cv_results_title = cross_validate(rf_clf_title, X_train_title, y_train_title, cv=cv, return_train_score=True)

In [ ]:
rf_cv_results_title

In [ ]:
print(f'Time for fitting classifier on the train set: {rf_cv_results_title["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {rf_cv_results_title["score_time"].mean()}')
print(f'Accuracy of Train: {rf_cv_results_title["train_score"].mean()}')
print(f'Accuracy of Validation: {rf_cv_results_title["test_score"].mean()}')

### 1.5 XGBoost

In [ ]:
from xgboost import XGBClassifier

xgboost_clf_title = Pipeline([
    ('scale', StandardScaler()),
    ('clf', XGBClassifier(random_state=1))
])

xgboost_cv_results_title = cross_validate(xgboost_clf_title, X_train_title, y_train_title, cv=cv, return_train_score=True)

In [ ]:
xgboost_cv_results_title

In [ ]:
print(f'Time for fitting classifier on the train set: {xgboost_cv_results_title["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {xgboost_cv_results_title["score_time"].mean()}')
print(f'Accuracy of Train: {xgboost_cv_results_title["train_score"].mean()}')
print(f'Accuracy of Validation: {xgboost_cv_results_title["test_score"].mean()}')

In [ ]:
results_title = pd.DataFrame(index=['Logistic Regression', 'Gaussian Naive Bayes', 'Support Vector Machine', 'Random Forest', 'XGBoost'])

results_title.loc['Logistic Regression', 'train_acc'] = lr_cv_results_title["train_score"].mean()
results_title.loc['Logistic Regression', 'val_acc'] = lr_cv_results_title["test_score"].mean()
results_title.loc['Logistic Regression', 'fit_time'] = lr_cv_results_title["fit_time"].mean()

results_title.loc['Gaussian Naive Bayes', 'train_acc'] = gnb_cv_results_title["train_score"].mean()
results_title.loc['Gaussian Naive Bayes', 'val_acc'] = gnb_cv_results_title["test_score"].mean()
results_title.loc['Gaussian Naive Bayes', 'fit_time'] = gnb_cv_results_title["fit_time"].mean()

results_title.loc['Support Vector Machine', 'train_acc'] = svm_cv_results_title["train_score"].mean()
results_title.loc['Support Vector Machine', 'val_acc'] = svm_cv_results_title["test_score"].mean()
results_title.loc['Support Vector Machine', 'fit_time'] = svm_cv_results_title["fit_time"].mean()

results_title.loc['Random Forest', 'train_acc'] = rf_cv_results_title["train_score"].mean()
results_title.loc['Random Forest', 'val_acc'] = rf_cv_results_title["test_score"].mean()
results_title.loc['Random Forest', 'fit_time'] = rf_cv_results_title["fit_time"].mean()

results_title.loc['XGBoost', 'train_acc'] = xgboost_cv_results_title["train_score"].mean()
results_title.loc['XGBoost', 'val_acc'] = xgboost_cv_results_title["test_score"].mean()
results_title.loc['XGBoost', 'fit_time'] = xgboost_cv_results_title["fit_time"].mean()

In [ ]:
results_title

# 2. Metadata and sentiments - all text

In [ ]:
X_train_text = X_train.iloc[:,5:20]
X_test_text = X_test.iloc[:,5:20]
y_train_text = y_train
y_test_text = y_test

In [ ]:
X_train_text

### 2.1 Logistic Regression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit


lr_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(random_state=1))
])

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=1)
lr_cv_results = cross_validate(lr_clf, X_train_text, y_train_text, cv=cv, return_train_score=True)

In [ ]:
lr_cv_results

In [ ]:
print(f'Time for fitting classifier on the train set: {lr_cv_results["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {lr_cv_results["score_time"].mean()}')
print(f'Accuracy of Train: {lr_cv_results["train_score"].mean()}')
print(f'Accuracy of Validation: {lr_cv_results["test_score"].mean()}')

### 2.2 Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import Normalizer

gnb_clf = Pipeline([
    ('scale', Normalizer()),
    ('clf', GaussianNB())
])

gnb_cv_results = cross_validate(gnb_clf, X_train_text, y_train_text, cv=cv, return_train_score=True)

In [ ]:
gnb_cv_results

In [ ]:
print(f'Time for fitting classifier on the train set: {gnb_cv_results["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {gnb_cv_results["score_time"].mean()}')
print(f'Accuracy of Train: {gnb_cv_results["train_score"].mean()}')
print(f'Accuracy of Validation: {gnb_cv_results["test_score"].mean()}')

### 2.3 SVM

In [ ]:
from sklearn.svm import SVC

svm_clf = Pipeline([
    ('scale', StandardScaler()),
    ('clf', SVC(random_state=1))
])

svm_cv_results = cross_validate(svm_clf, X_train_text, y_train_text, cv=cv, return_train_score=True)

In [ ]:
svm_cv_results

In [ ]:
print(f'Time for fitting classifier on the train set: {svm_cv_results["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {svm_cv_results["score_time"].mean()}')
print(f'Accuracy of Train: {svm_cv_results["train_score"].mean()}')
print(f'Accuracy of Validation: {svm_cv_results["test_score"].mean()}')

### 2.4 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = Pipeline([
    ('scale', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=1))
])

rf_cv_results = cross_validate(rf_clf, X_train_text, y_train_text, cv=cv, return_train_score=True)

In [ ]:
rf_cv_results

In [ ]:
print(f'Time for fitting classifier on the train set: {rf_cv_results["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {rf_cv_results["score_time"].mean()}')
print(f'Accuracy of Train: {rf_cv_results["train_score"].mean()}')
print(f'Accuracy of Validation: {rf_cv_results["test_score"].mean()}')

### 2.5 XGBoost

In [ ]:
from xgboost import XGBClassifier

xgboost_clf = Pipeline([
    ('scale', StandardScaler()),
    ('clf', XGBClassifier(random_state=1))
])

xgboost_cv_results = cross_validate(xgboost_clf, X_train_text, y_train_text, cv=cv, return_train_score=True)

In [ ]:
xgboost_cv_results

In [ ]:
print(f'Time for fitting classifier on the train set: {xgboost_cv_results["fit_time"].mean()}')
print(f'Time for scoring classifier on the validation set: {xgboost_cv_results["score_time"].mean()}')
print(f'Accuracy of Train: {xgboost_cv_results["train_score"].mean()}')
print(f'Accuracy of Validation: {xgboost_cv_results["test_score"].mean()}')

In [ ]:
results_all_text = pd.DataFrame(index=['Logistic Regression', 'Gaussian Naive Bayes', 'Support Vector Machine', 'Random Forest', 'XGBoost'])

results_all_text.loc['Logistic Regression', 'train_acc'] = lr_cv_results["train_score"].mean()
results_all_text.loc['Logistic Regression', 'val_acc'] = lr_cv_results["test_score"].mean()
results_all_text.loc['Logistic Regression', 'fit_time'] = lr_cv_results["fit_time"].mean()

results_all_text.loc['Gaussian Naive Bayes', 'train_acc'] = gnb_cv_results["train_score"].mean()
results_all_text.loc['Gaussian Naive Bayes', 'val_acc'] = gnb_cv_results["test_score"].mean()
results_all_text.loc['Gaussian Naive Bayes', 'fit_time'] = gnb_cv_results["fit_time"].mean()

results_all_text.loc['Support Vector Machine', 'train_acc'] = svm_cv_results["train_score"].mean()
results_all_text.loc['Support Vector Machine', 'val_acc'] = svm_cv_results["test_score"].mean()
results_all_text.loc['Support Vector Machine', 'fit_time'] = svm_cv_results["fit_time"].mean()

results_all_text.loc['Random Forest', 'train_acc'] = rf_cv_results["train_score"].mean()
results_all_text.loc['Random Forest', 'val_acc'] = rf_cv_results["test_score"].mean()
results_all_text.loc['Random Forest', 'fit_time'] = rf_cv_results["fit_time"].mean()

results_all_text.loc['XGBoost', 'train_acc'] = xgboost_cv_results["train_score"].mean()
results_all_text.loc['XGBoost', 'val_acc'] = xgboost_cv_results["test_score"].mean()
results_all_text.loc['XGBoost', 'fit_time'] = xgboost_cv_results["fit_time"].mean()

In [ ]:
results_all_text

# 3. Display results

In [ ]:
from IPython.display import display_html 

results_title_style = results_title.style.set_table_attributes("style='display:inline; margin-right:20px;'").set_caption("Title Only")
results_all_text_style = results_all_text.style.set_table_attributes("style='display:inline'").set_caption("Title & Text")

display_html(results_title_style._repr_html_() + results_all_text_style._repr_html_(), raw=True)

1. SVM takes the longest to fit across both experiments
2. For title only, although SVM performed the best, XGBoost wasn't far off in terms of validation accuracy and took 6.5 times less to fit
3. For all text, although SVM performed the best, the long fitting time dissauades us from using it, instead XGBoost or even Random Forest would be a better choice
4. On average, validation accuracy for title is higher than that of all text, hence we could say that we are able to predict if an article is real based on the title alone

# 4. Test Accuracy - Title

In [ ]:
results_title_final = pd.DataFrame(index=['Logistic Regression', 'Gaussian Naive Bayes', 'Support Vector Machine', 'Random Forest', 'XGBoost'])

In [ ]:
lr_clf_title = sklearn.base.clone(lr_clf_title)
lr_clf_title.fit(X_train_title, y_train_title)
lr_clf_title_train_acc = lr_clf_title.score(X_train_title, y_train_title)
lr_clf_title_test_acc = lr_clf_title.score(X_test_title, y_test_title)
results_title_final.loc['Logistic Regression', 'train_acc'] = lr_clf_title_train_acc
results_title_final.loc['Logistic Regression', 'test_acc'] = lr_clf_title_test_acc

gnb_clf_title = sklearn.base.clone(gnb_clf_title)
gnb_clf_title.fit(X_train_title, y_train_title)
gnb_clf_title_train_acc = gnb_clf_title.score(X_train_title, y_train_title)
gnb_clf_title_test_acc = gnb_clf_title.score(X_test_title, y_test_title)
results_title_final.loc['Gaussian Naive Bayes', 'train_acc'] = gnb_clf_title_train_acc
results_title_final.loc['Gaussian Naive Bayes', 'test_acc'] = gnb_clf_title_test_acc

svm_clf_title = sklearn.base.clone(svm_clf_title)
svm_clf_title.fit(X_train_title, y_train_title)
svm_clf_title_train_acc = svm_clf_title.score(X_train_title, y_train_title)
svm_clf_title_test_acc = svm_clf_title.score(X_test_title, y_test_title)
results_title_final.loc['Support Vector Machine', 'train_acc'] = svm_clf_title_train_acc
results_title_final.loc['Support Vector Machine', 'test_acc'] = svm_clf_title_test_acc

rf_clf_title = sklearn.base.clone(rf_clf_title)
rf_clf_title.fit(X_train_title, y_train_title)
rf_clf_title_train_acc = rf_clf_title.score(X_train_title, y_train_title)
rf_clf_title_test_acc = rf_clf_title.score(X_test_title, y_test_title)
results_title_final.loc['Random Forest', 'train_acc'] = rf_clf_title_train_acc
results_title_final.loc['Random Forest', 'test_acc'] = rf_clf_title_test_acc

xgboost_clf_title = sklearn.base.clone(xgboost_clf_title)
xgboost_clf_title.fit(X_train_title, y_train_title)
xgboost_clf_title_train_acc = xgboost_clf_title.score(X_train_title, y_train_title)
xgboost_clf_title_test_acc = xgboost_clf_title.score(X_test_title, y_test_title)
results_title_final.loc['XGBoost', 'train_acc'] = xgboost_clf_title_train_acc
results_title_final.loc['XGBoost', 'test_acc'] = xgboost_clf_title_test_acc

results_title_final


# 5. Test Accuracy - Title + Text

In [ ]:
results_all_final = pd.DataFrame(index=['Logistic Regression', 'Gaussian Naive Bayes', 'Support Vector Machine', 'Random Forest', 'XGBoost'])

In [ ]:
lr_clf = sklearn.base.clone(lr_clf)
lr_clf.fit(X_train_text, y_train_text)
lr_all_train_acc = lr_clf.score(X_train_text, y_train_text)
lr_all_test_acc = lr_clf.score(X_test_text, y_test_text)
results_all_final.loc['Logistic Regression', 'train_acc'] = lr_all_train_acc
results_all_final.loc['Logistic Regression', 'test_acc'] = lr_all_test_acc

gnb_clf = sklearn.base.clone(gnb_clf)
gnb_clf.fit(X_train_text, y_train_text)
gnb_all_train_acc = gnb_clf.score(X_train_text, y_train_text)
gnb_all_test_acc = gnb_clf.score(X_test_text, y_test_text)
results_all_final.loc['Gaussian Naive Bayes', 'train_acc'] = gnb_all_train_acc
results_all_final.loc['Gaussian Naive Bayes', 'test_acc'] = gnb_all_test_acc

svm_clf = sklearn.base.clone(svm_clf)
svm_clf.fit(X_train_text, y_train_text)
svm_all_train_acc = svm_clf.score(X_train_text, y_train_text)
svm_all_test_acc = svm_clf.score(X_test_text, y_test_text)
results_all_final.loc['Support Vector Machine', 'train_acc'] = svm_all_train_acc
results_all_final.loc['Support Vector Machine', 'test_acc'] = svm_all_test_acc

rf_clf = sklearn.base.clone(rf_clf)
rf_clf.fit(X_train_text, y_train_text)
rf_all_train_acc = rf_clf.score(X_train_text, y_train_text)
rf_all_test_acc = rf_clf.score(X_test_text, y_test_text)
results_all_final.loc['Random Forest', 'train_acc'] = rf_all_train_acc
results_all_final.loc['Random Forest', 'test_acc'] = rf_all_test_acc

xgboost_clf = sklearn.base.clone(xgboost_clf)
xgboost_clf.fit(X_train_text, y_train_text)
xgboost_all_train_acc = xgboost_clf.score(X_train_text, y_train_text)
xgboost_all_test_acc = xgboost_clf.score(X_test_text, y_test_text)
results_all_final.loc['XGBoost', 'train_acc'] = xgboost_all_train_acc
results_all_final.loc['XGBoost', 'test_acc'] = xgboost_all_test_acc

results_all_final

In [ ]:
results_title_style = results_title_final.style.set_table_attributes("style='display:inline; margin-right:20px;'").set_caption("Title Only")
results_all_text_style = results_all_final.style.set_table_attributes("style='display:inline'").set_caption("Title & Text")

display_html(results_title_style._repr_html_() + results_all_text_style._repr_html_(), raw=True)

# 3. Tfidf - all text

In [ ]:
X_train_vec = X_train['all_text']
X_test_vec = X_test['all_text']

In [ ]:
X_train_vec

In [ ]:
# Converting text to vectors

from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
X_train_vec = vectorization.fit_transform(X_train_vec)
X_test_vec = vectorization.transform(X_test_vec)

In [ ]:
nlp_results = {}

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(random_state=1)
GBC.fit(X_train_vec, y_train)

print(f'Accuracy of Train: {GBC.score(X_train_vec, y_train)}')
print(f'Accuracy of Test: {GBC.score(X_test_vec, y_test)}')

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(X_train_vec, y_train)
nlp_results['LR'] = [LR.score(X_train_vec, y_train), LR.score(X_test_vec, y_test)] 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB().fit(X_train_vec, y_train)
nlp_results['NB']= [NB.score(X_train_vec, y_train), NB.score(X_test_vec, y_test)]

In [ ]:
from sklearn.svm import SVC

SVM = SVC(kernel='rbf', gamma='scale', random_state=1)
SVM.fit(X_train_vec, y_train)
nlp_results['SVM']= [SVM.score(X_train_vec, y_train), SVM.score(X_test_vec, y_test)]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=1)
RFC.fit(X_train_vec, y_train)

nlp_results['RFC']= [RFC.score(X_train_vec, y_train), RFC.score(X_test_vec, y_test)]

In [ ]:
from xgboost import XGBClassifier
XGB = XGBClassifier(random_state=1)
XGB.fit(X_train_vec, y_train)
nlp_results['XGB'] = [XGB.score(X_train_vec, y_train), XGB.score(X_test_vec, y_test)]

In [ ]:
print(nlp_results)

### Results for all text

In [ ]:
import pandas as pd
tfidf_results_all_text = pd.DataFrame()

tfidf_results_all_text.loc['Logistic Regression', 'train_acc'] = nlp_results['LR'][0]
tfidf_results_all_text.loc['Logistic Regression', 'test_acc'] = nlp_results['LR'][1]

tfidf_results_all_text.loc['Multinomial Naive Bayes', 'train_acc'] = nlp_results['NB'][0]
tfidf_results_all_text.loc['Multinomial Naive Bayes', 'test_acc'] = nlp_results['NB'][1]

tfidf_results_all_text.loc['Support Vector Machine', 'train_acc'] = nlp_results['SVM'][0]
tfidf_results_all_text.loc['Support Vector Machine', 'test_acc'] = nlp_results['SVM'][1]

tfidf_results_all_text.loc['Random Forest', 'train_acc'] = nlp_results['RFC'][0]
tfidf_results_all_text.loc['Random Forest', 'test_acc'] = nlp_results['RFC'][1]

tfidf_results_all_text.loc['XGBoost', 'train_acc'] = nlp_results['XGB'][0]
tfidf_results_all_text.loc['XGBoost', 'test_acc'] = nlp_results['XGB'][1]
tfidf_results_all_text

# 4. Tfidf - title only

In [ ]:
X_train_vec2 = X_train['title']
X_test_vec2 = X_test['title']

In [ ]:
X_train_vec2.head()

In [ ]:
# Converting text to vectors

from sklearn.feature_extraction.text import TfidfVectorizer

vectorization2 = TfidfVectorizer()
X_train_vec2 = vectorization2.fit_transform(X_train_vec2)
X_test_vec2 = vectorization2.transform(X_test_vec2)

In [ ]:
nlp_results_title = {}

In [ ]:
LR2 = LogisticRegression()
LR2.fit(X_train_vec2, y_train)
nlp_results_title['LR'] = [LR2.score(X_train_vec2, y_train), LR2.score(X_test_vec2, y_test)] 

NB2 = MultinomialNB().fit(X_train_vec2, y_train)
nlp_results_title['NB']= [NB2.score(X_train_vec2, y_train), NB2.score(X_test_vec2, y_test)]

In [ ]:
RFC2 = RandomForestClassifier(random_state=1)
RFC2.fit(X_train_vec2, y_train)
nlp_results_title['RFC']= [RFC2.score(X_train_vec2, y_train), RFC2.score(X_test_vec2, y_test)]

In [ ]:
XGB2 = XGBClassifier(random_state=1)
XGB2.fit(X_train_vec2, y_train)
nlp_results_title['XGB'] = [XGB2.score(X_train_vec2, y_train), XGB2.score(X_test_vec2, y_test)]

In [ ]:
from sklearn.svm import SVC

SVM2 = SVC(kernel='rbf', gamma='scale', random_state=1)
SVM2.fit(X_train_vec2, y_train)
nlp_results_title['SVM']= [SVM2.score(X_train_vec2, y_train), SVM2.score(X_test_vec2, y_test)]

In [ ]:
print(nlp_results_title)

### Results for title only

In [ ]:
import pandas as pd
tfidf_results_title = pd.DataFrame()

tfidf_results_title.loc['Logistic Regression', 'train_acc'] = nlp_results_title['LR'][0]
tfidf_results_title.loc['Logistic Regression', 'test_acc'] = nlp_results_title['LR'][1]

tfidf_results_title.loc['Multinomial Naive Bayes', 'train_acc'] = nlp_results_title['NB'][0]
tfidf_results_title.loc['Multinomial Naive Bayes', 'test_acc'] = nlp_results_title['NB'][1]

tfidf_results_title.loc['Support Vector Machine', 'train_acc'] = nlp_results_title['SVM'][0]
tfidf_results_title.loc['Support Vector Machine', 'test_acc'] = nlp_results_title['SVM'][1]

tfidf_results_title.loc['Random Forest', 'train_acc'] = nlp_results_title['RFC'][0]
tfidf_results_title.loc['Random Forest', 'test_acc'] = nlp_results_title['RFC'][1]

tfidf_results_title.loc['XGBoost', 'train_acc'] = nlp_results_title['XGB'][0]
tfidf_results_title.loc['XGBoost', 'test_acc'] = nlp_results_title['XGB'][1]
tfidf_results_title

## Hyperparameter tuning XGBoost

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
params = {'max_depth': [3, 6, 10, 15],
      'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
      'subsample': np.arange(0.5, 1.0, 0.1),
      'colsample_bytree': np.arange(0.5, 1.0, 0.1),
      'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
      }
search = RandomizedSearchCV(estimator=XGB,
                         param_distributions=params,
                         scoring='accuracy',
                         n_jobs=2, random_state=1)
search.fit(X_train_vec, y_train)
print(search.best_score_)
search.best_params_

In [ ]:
XGB_tuned = XGBClassifier(**search.best_params_, random_state=1)
XGB_tuned.fit(X_train_vec, y_train)
nlp_results['XGB Tuned'] = [XGB_tuned.score(X_train_vec, y_train), XGB_tuned.score(X_test_vec, y_test)]
print("Training Accuracy:", nlp_results['XGB Tuned'][0])
print("Testing Accuracy:", nlp_results['XGB Tuned'][1])

Accuracy went up by a marginal 0.02% after tuning hyperparameters. Hence, hyperparameter tuning may not be worth the efforts.

In [ ]:
y_pred_prob = XGB.predict_proba(X_test_vec)[:,1]
y_pred_prob2 = XGB_tuned.predict_proba(X_test_vec)[:,1]
print('XGB ROC_AUC:', metrics.roc_auc_score(y_test, y_pred_prob))
print('XGB Tuned ROC_AUC:', metrics.roc_auc_score(y_test, y_pred_prob2))